In [25]:
import yfinance as yf
import talib
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import datetime

# Descargar datos de precios de Bitcoin (BTC-USD) de Yahoo Finance
btc = yf.download('BTC-USD',period="moy", interval='5m')

# Calcular el promedio móvil de n períodos de alta y baja
avg_high = talib.SMA(btc['High'].values, timeperiod=3)
avg_low = talib.SMA(btc['Low'].values, timeperiod=3)

# Calcular el CE utilizando la fórmula
ce = btc['Close'].shift(1) - (avg_high + avg_low) / 2

# Calcular el RSI utilizando la biblioteca Ta-Lib
rsi = talib.RSI(btc['Close'].values, timeperiod=14)

# Calcular medias móviles exponenciales de 8 y 200 períodos
ema_8 = talib.EMA(btc['Close'].values, timeperiod=8)
ema_200 = talib.EMA(btc['Close'].values, timeperiod=200)

# Generar una alerta cuando el precio está en un punto de compra o venta, hay una divergencia en el RSI y cruce de medias móviles
alerts = []
for i, val in enumerate(ce):
    if val > 0 and btc['Close'][i] > btc['Close'][i-1] and rsi[i] < rsi[i-1] and ema_8[i] > ema_200[i]:
        alerts.append((i, "Compra", btc['Close'][i], rsi[i]))
    elif val < 0 and btc['Close'][i] < btc['Close'][i-1] and rsi[i] > rsi[i-1] and ema_8[i] < ema_200[i]:
        alerts.append((i, "Venta", btc['Close'][i], rsi[i]))

# Calcular la ganancia total
profit = 0
for i, (period, type, price, rsi) in enumerate(alerts):
    if type == "Compra":
        profit += price
    elif type == "Venta":
        profit -= price

# Imprimir las alertas generadas
print("Alertas generadas:")
for i, (period, type, price, rsi) in enumerate(alerts):
    print(f"Período {period}: {type} en precio {price}, RSI {rsi}")

# Imprimir la ganancia total
print(f"Ganancia total: {profit}")

[*********************100%***********************]  1 of 1 completed
Alertas generadas:
Ganancia total: 0
